In [ ]:
###Import standard packages###
import numpy as np
from scipy import io

###Import functions for binning data for preprocessing###
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output
import pickle

import matplotlib.pyplot as plt
import pandas as pd
from Area2_analysis.multi_area_funcs import smooth_spk, fit_and_predict, sub_and_predict, mp_fit_lag_r2, mp_sub_lag_r2
import multiprocessing as mp

## S1

In [ ]:
# foldername = "/Users/sherryan/area2_population_analysis/multi_area/"
# filename = foldername + "s1_data_raw.mat"
# s1_data_raw = io.loadmat(filename)

# spike_times=s1_data_raw['spike_times']
# acc = s1_data_raw['acc']
# vel = s1_data_raw['vels']
# times = s1_data_raw['vel_times']

# #original data has 0.01 s = 10 ms bins 
# dt = 0.01
# t_start = times[0]
# t_end = times[-1]

# spike_times = np.squeeze(spike_times)
# for i in range(spike_times.shape[0]):
#     spike_times[i]=np.squeeze(spike_times[i])

# neural_data=bin_spikes(spike_times,dt,t_start,t_end)
# gauss_width = 40 #in ms
# bin_width = dt*1000
# smth_40 = smooth_spk(neural_data, gauss_width, bin_width)
# smth_20 = smooth_spk(neural_data, 20, bin_width)

# accs_binned=bin_output(acc,times,dt,t_start,t_end)
# vels_binned=bin_output(vel,times,dt,t_start,t_end)

# with open(foldername+'proc_data_s1.pickle','wb') as f:
#     pickle.dump([smth_40,smth_20,accs_binned,vels_binned],f)

In [ ]:
folder="/Users/sherryan/area2_population_analysis/multi_area/"

with open(folder+'proc_data_s1.pickle','rb') as f:
    s1_smth_40,s1_smth_20,s1_accs_binned, s1_vels_binned=pickle.load(f,encoding='latin1')
print(s1_smth_40.shape)
print(s1_smth_20.shape)
print(s1_accs_binned.shape)
print(s1_vels_binned.shape)

n_neurons = s1_smth_20.shape[1]

In [ ]:
figDir = "/Users/sherryan/area2_population_analysis/figures/multi/neurons/S1/"
dim = n_neurons
x = s1_smth_40
y_field = 'hand_acc'
if y_field == 'hand_vel':
    y = s1_vels_binned
if y_field == 'hand_acc':
    y = s1_accs_binned
lag_axis = np.arange(-300,300,20)
curr_r2_array = [pool.apply(mp_fit_lag_r2, args=[x,y,lag,10]) for lag in lag_axis]

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting ' + y_field )
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_0.png', dpi = 'figure')
plt.close()

_, weights, _ = fit_and_predict(x,y,time_max,bin_size=10)
#subtract predictions with primary decoding dimensions (at time with max R2)
sub_r2_array = [pool.apply(mp_sub_lag_r2, args=[x,y,lag,10,weights]) for lag in lag_axis]

plt.plot(lag_axis,sub_r2_array)
plt.title('R2 score projecting out #1 t_max dim')
idx_max = np.argmax(sub_r2_array)
time_max = lag_axis[idx_max]
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_1.png', dpi = 'figure')
plt.close()


## M1

In [ ]:
# foldername = "/Users/sherryan/area2_population_analysis/multi_area/"
# filename = foldername + "m1_data_raw.mat"
# m1_data_raw = io.loadmat(filename)

# spike_times=m1_data_raw['spike_times'][:70,:] #only first 70 are M1 neurons
# acc = m1_data_raw['acc']
# vel = m1_data_raw['vels']
# times = m1_data_raw['vel_times']

# #original data has 0.001 s = 1 ms bins 
# dt = 0.01
# t_start = times[0]
# t_end = times[-1]

# spike_times = np.squeeze(spike_times)
# for i in range(spike_times.shape[0]):
#     spike_times[i]=np.squeeze(spike_times[i])

# neural_data=bin_spikes(spike_times,dt,t_start,t_end)
# gauss_width = 40 #in ms
# bin_width = dt*1000
# smth_40 = smooth_spk(neural_data, gauss_width, bin_width)
# smth_20 = smooth_spk(neural_data, 20, bin_width)

# accs_binned=bin_output(acc,times,dt,t_start,t_end)
# vels_binned=bin_output(vel,times,dt,t_start,t_end)

# with open(foldername+'proc_data_m1.pickle','wb') as f:
#     pickle.dump([smth_40, smth_20,accs_binned,vels_binned],f)

In [ ]:
folder="/Users/sherryan/area2_population_analysis/multi_area/"

with open(folder+'proc_data_m1.pickle','rb') as f:
    m1_smth_40,m1_smth_20,m1_accs_binned, m1_vels_binned=pickle.load(f,encoding='latin1')
print(m1_smth_40.shape)
print(m1_smth_20.shape)
print(m1_accs_binned.shape)
print(m1_vels_binned.shape)

n_neurons = m1_smth_20.shape[1]

In [ ]:
figDir = "/Users/sherryan/area2_population_analysis/figures/multi/neurons/M1/"
dim = n_neurons
x = m1_smth_40
y_field = 'hand_vel'
if y_field == 'hand_vel':
    y = m1_vels_binned
if y_field == 'hand_acc':
    y = m1_accs_binned
lag_axis = np.arange(-300,300,20)
pool = mp.Pool(mp.cpu_count())
curr_r2_array = [pool.apply(mp_fit_lag_r2, args=[x,y,lag,10]) for lag in lag_axis]
pool.close()

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting ' + y_field )
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_0.png', dpi = 'figure')
plt.close()

_, weights, _ = fit_and_predict(x,y,time_max,bin_size=10)
#subtract predictions with primary decoding dimensions (at time with max R2)
pool = mp.Pool(mp.cpu_count())
sub_r2_array = [pool.apply(mp_sub_lag_r2, args=[x,y,lag,10,weights]) for lag in lag_axis]
pool.close()
plt.plot(lag_axis,sub_r2_array)
plt.title('R2 score projecting out #1 t_max dim')
idx_max = np.argmax(sub_r2_array)
time_max = lag_axis[idx_max]
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_1.png', dpi = 'figure')
plt.close()
